데이터 전처리

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath("../"))

from utils import *
from MLP import train_and_evaluate
from permutation import permutation

# Load dataset
region="daegu"
path = f"../../collect_data/filtered/kma/merged/kma_{region}_meta.csv"

# 2. CSV 파일 읽어서 데이터프레임으로 변환
korea_data = pd.read_csv(path)
china_data = concat_china_data(korea_data)

korea_data = convert_timesteps(korea_data)
print_missing_info(korea_data)

china_data = convert_timesteps(china_data)
print_missing_info(china_data)

(40177, 22)
Series([], dtype: float64)
Index(['TM', 'STN', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'PV', 'WW',
       'CA_TOT', 'CA_MID', 'CT', 'VS', 'TS', 'IR', 'PM10', '지점', '위도', '경도',
       'diff'],
      dtype='object')
(9486, 107)
VV             0.031626
WD_yanan       0.021084
WS_yanan       0.021084
TA_yanan       0.105419
TD_yanan       0.274088
HM_yanan       0.685220
PS_yanan       0.031626
PR             0.063251
VV_tongliao    0.010542
WD_tongliao    0.010542
WS_tongliao    0.010542
TA_tongliao    0.137044
TD_tongliao    0.253004
HM_tongliao    0.242463
PS_tongliao    0.010542
PR_tongliao    0.010542
VV_qingdao     0.094877
WD_qingdao     0.115960
WS_qingdao     0.115960
TA_qingdao     0.094877
TD_qingdao     0.200295
HM_qingdao     0.790639
PS_qingdao     0.094877
PR_qingdao     0.094877
VV_chifeng     0.063251
WD_chifeng     0.021084
WS_chifeng     0.063251
TA_chifeng     0.115960
TD_chifeng     0.263546
HM_chifeng     0.179211
PS_chifeng     0.031626
PR_chifeng     0

MLP-Model - 한국만

In [2]:
removed_columns=['TM', 'PM10','WW','CT','diff']

# Features (X) and target (y)
X = korea_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = korea_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)



Cross-validated Mean Absolute Error: 12.187485504150391
252/252 [==============================] - 0s 589us/step - loss: 212.9158 - mae: 10.4678
Test Set Mean Absolute Error: 10.467801094055176


질적 분석

In [3]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features_korea, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 9852.8955078125
252/252 [==============================] - 0s 557us/step
Feature: STN, Importance: 0.0
Feature: WD, Importance: -0.23620871599323437
Feature: WS, Importance: -0.22208909090495582
Feature: PA, Importance: 144.37799951554115
Feature: PS, Importance: 104.35264658579926
Feature: TA, Importance: 8.993689303172868
Feature: TD, Importance: -0.5272392977889467
Feature: HM, Importance: -1.833927386637697
Feature: PV, Importance: -1.9369047314237833
Feature: CA_TOT, Importance: -0.08761137311939819
Feature: CA_MID, Importance: -0.014147353967746312
Feature: VS, Importance: 253.05799266799386
Feature: TS, Importance: -2.7960245836457034
Feature: IR, Importance: -0.18872312613293615
Feature: 지점, Importance: 0.0
Feature: 위도, Importance: 0.0
Feature: 경도, Importance: 0.0

Top 10 Important Features (Sorted Descending by Importance Score):
Feature: VS, Importance: 253.05799266799386
Feature: PA, Importance: 144.37799951554115
Feature: PS, Importan

MLP model - 중국 데이터 포함

In [4]:
removed_columns=['TM', 'PM10','WW','CT','STN_dalian','STN_tongliao','STN_qingdao','STN_chifeng','diff']

# Features (X) and target (y)
X = china_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = china_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 12.854205513000489
60/60 [==============================] - 0s 610us/step - loss: 202.9594 - mae: 10.2789
Test Set Mean Absolute Error: 10.278911590576172


질적 분석

In [5]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features_china, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 14218.39453125
60/60 [==============================] - 0s 545us/step
Feature: STN, Importance: 0.0
Feature: WD, Importance: -0.01790536090684327
Feature: WS, Importance: -0.00480802674737788
Feature: PA, Importance: 0.6747796367160845
Feature: PS, Importance: 0.014606706560334715
Feature: TA, Importance: 0.14396610802614304
Feature: TD, Importance: 0.011108089321714943
Feature: HM, Importance: -0.24021883041059483
Feature: PV, Importance: 0.003990747352872859
Feature: CA_TOT, Importance: 0.00028119906164647546
Feature: CA_MID, Importance: -0.016392628034373045
Feature: VS, Importance: 20.517519727571745
Feature: TS, Importance: -0.007127375548043347
Feature: IR, Importance: 4.655136508517899e-05
Feature: 지점, Importance: 0.0
Feature: 위도, Importance: 0.0
Feature: 경도, Importance: 0.0
Feature: STN_yanan, Importance: 0.0
Feature: IW, Importance: 0.0
Feature: IR_yanan, Importance: 0.0
Feature: IX, Importance: 0.0
Feature: VV, Importance: 0.01024557099

## MLP 한국만
Feature Importance 음수인 feature 제거한 버전

In [6]:
print(low_importance_features_korea)

['STN', 'WD', 'WS', 'TD', 'HM', 'PV', 'CA_TOT', 'CA_MID', 'TS', 'IR', '지점', '위도', '경도']


In [7]:
removed_columns=['TM', 'PM10','WW','CT','diff']
removed_columns+=low_importance_features_korea

# Features (X) and target (y)
X = korea_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = korea_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 14.214463806152343
252/252 [==============================] - 0s 638us/step - loss: 315.1276 - mae: 13.7785
Test Set Mean Absolute Error: 13.778451919555664


In [8]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 8160.79052734375
252/252 [==============================] - 0s 491us/step
Feature: PA, Importance: 96.15424279700255
Feature: PS, Importance: 18.48109668802499
Feature: TA, Importance: -0.30090520755147737
Feature: VS, Importance: 114.73303460273856

Top 10 Important Features (Sorted Descending by Importance Score):
Feature: VS, Importance: 114.73303460273856
Feature: PA, Importance: 96.15424279700255
Feature: PS, Importance: 18.48109668802499
Feature: TA, Importance: -0.30090520755147737


## MLP 중국포함
Feature Importance 음수인 feature 제거한 버전

In [9]:
print(low_importance_features_china)

['STN', 'WD', 'WS', 'HM', 'CA_MID', 'TS', '지점', '위도', '경도', 'STN_yanan', 'IW', 'IR_yanan', 'IX', 'WD_yanan', 'WS_yanan', 'TD_yanan', 'HM_yanan', 'PS_yanan', 'RH', 'LON', 'LAT', 'IW_tongliao', 'IR_tongliao', 'IX_tongliao', 'WD_tongliao', 'WS_tongliao', 'HM_tongliao', 'PS_tongliao', 'PT_tongliao', 'PR_tongliao', 'RH_tongliao', 'PM10_tongliao', 'LON_tongliao', 'LAT_tongliao', 'IW_qingdao', 'IR_qingdao', 'IX_qingdao', 'WD_qingdao', 'WS_qingdao', 'TA_qingdao', 'HM_qingdao', 'PS_qingdao', 'PR_qingdao', 'RH_qingdao', 'LON_qingdao', 'LAT_qingdao', 'IW_chifeng', 'IR_chifeng', 'IX_chifeng', 'WS_chifeng', 'TA_chifeng', 'HM_chifeng', 'PS_chifeng', 'PR_chifeng', 'RH_chifeng', 'LON_chifeng', 'LAT_chifeng', 'IW_dalian', 'IR_dalian', 'IX_dalian', 'VV_dalian', 'WD_dalian', 'PT_dalian', 'RH_dalian', 'LON_dalian', 'LAT_dalian']


In [10]:
removed_columns=['TM', 'PM10','WW','CT','STN_dalian','STN_tongliao','STN_qingdao','STN_chifeng','diff']
removed_columns+=low_importance_features_china

# Features (X) and target (y)
X = china_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = china_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 12.909667205810546
60/60 [==============================] - 0s 646us/step - loss: 208.1641 - mae: 10.5375
Test Set Mean Absolute Error: 10.537544250488281


In [11]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 11801.77734375
60/60 [==============================] - 0s 503us/step
Feature: PA, Importance: 74.91520069512316
Feature: PS, Importance: 37.10874963766473
Feature: TA, Importance: 7.212350287101617
Feature: TD, Importance: -0.2635136980645257
Feature: PV, Importance: -0.44989364044377
Feature: CA_TOT, Importance: -0.11481410128599237
Feature: VS, Importance: 117.04253437004118
Feature: IR, Importance: -0.046274990728306875
Feature: VV, Importance: -0.06681350847593422
Feature: TA_yanan, Importance: -0.015372520106757293
Feature: PT, Importance: 0.00311417956600053
Feature: PR, Importance: 0.001091168929997366
Feature: PM10_yanan, Importance: 0.30629007702737
Feature: VV_tongliao, Importance: 0.010873523729060253
Feature: TA_tongliao, Importance: 0.002385509275700315
Feature: TD_tongliao, Importance: 0.8928546236717011
Feature: VV_qingdao, Importance: 0.008389189070840075
Feature: TD_qingdao, Importance: 0.13184558915072558
Feature: PT_qingdao, I